In [1]:
import sympy
from sympy import Rational
import numpy as np
import pandas as pd
import math
from gurobipy import *

train_info = pd.read_csv(r'D:/学习/课题/inverse optimization data/train_info_multi_class_multi_origin.csv')
commuter_data = pd.read_csv(r'D:/学习/课题/inverse optimization data/multi_class_multi_origin_commuter_data.csv')

#变量维度:车次，站间区间
di = len(train_info['TYPE'])
station_interval = list(range(151019789, 151019778, -1))

alpha_value_initial = 0.1282 
beta_value_initial = 0.1476 
gamma_value_initial = 0.1545 
g0_value_initial = 0.0042 
theta_value_initial = 0.011 

#上层决策变量值记录
upper_parameters_record = sympy.Matrix([alpha_value_initial, beta_value_initial, gamma_value_initial, g0_value_initial, theta_value_initial])
#上层目标值记录
upper_obj_record = []
#收敛准侧值记录
indicator_record = []

origins_input = [151019789, 151019787, 151019785, 151019784]
origins_interval_num = {151019789:11, 151019787:9, 151019785:7, 151019784:6}
userclass_input = {origins_input[0]:[0, 1, 2, 3], origins_input[1]:[0, 1, 2, 3, 4, 5], origins_input[2]:[0, 1, 2, 3, 4], origins_input[3]:[0, 1, 2, 3, 4]}

train_choice_set_input = {}
train_choice_set_input[origins_input[0]] = {0:list(range(11,30)), 1:list(range(5,30)), 2:list(range(19,45)), 3:list(range(16,40))}
train_choice_set_input[origins_input[1]] = {0:list(range(14,33)), 1:list(range(11,30)), 2:list(range(21,43)), 3:list(range(21,40)), 4:list(range(24,45)), 5:list(range(3,25))}
train_choice_set_input[origins_input[2]] = {0:list(range(26,45)), 1:list(range(2,30)), 2:list(range(19,45)), 3:list(range(14,33)), 4:list(range(11,30))}
train_choice_set_input[origins_input[3]] = {0:list(range(14,27)), 1:list(range(24,45)), 2:list(range(16,33)), 3:list(range(21,40)), 4:list(range(1,25))}

demand_input = {}
demand_input[origins_input[0]] = {0: 152, 1: 94, 2: 133, 3: 115}
demand_input[origins_input[1]] = {0: 99, 1: 128, 2: 58, 3: 60, 4: 69, 5: 116}
demand_input[origins_input[2]] = {0: 59, 1: 122, 2: 90, 3: 85, 4: 72}
demand_input[origins_input[3]] = {0: 102, 1: 81, 2: 124, 3: 112, 4: 131}

desired_arrival_time_input = {}
desired_arrival_time_input[origins_input[0]] = {0:30900, 1:29700, 2:33300, 3:32100}
desired_arrival_time_input[origins_input[1]] = {0:32100, 1:30900, 2:33300, 3:32700, 4:34500, 5:29700}
desired_arrival_time_input[origins_input[2]] = {0:34500, 1:30300, 2:32700, 3:31500, 4:30900}
desired_arrival_time_input[origins_input[3]] = {0:30900, 1:34500, 2:31500, 3:32700, 4:29700}

#出行时间
T = {}
for o in origins_input:
    for j in train_info['TYPE']:
        T[(o,j)] = train_info[train_info['TYPE'] == j]['TRIP_TIME_average_%d'%o].iloc[0]
        
#乘车时间
Tv = {151019789: 110, 151019788: 130, 151019787: 140, 151019786: 120, 151019785: 140, 151019784: 190, 151019783: 180, 151019782: 180, 151019781: 190, 151019780: 190, 151019779: 140}

#n0既有流量
n0 = {}
for j in train_info['TYPE']:
    for i in station_interval:
        n0[(j,i)] = train_info[train_info['TYPE'] == j]['%d'%i].values[0]

#TE和TL
TE = {}
TL = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            TE[(o,i,j)] = max(desired_arrival_time_input[o][i] - train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0], 0)
            TL[(o,i,j)] = max(train_info[train_info['TYPE'] == j]['ARRIVETIME_average_%d'%o].values[0] - desired_arrival_time_input[o][i], 0)

In [2]:
#定义符号
alpha = sympy.Symbol('alpha')
beta = sympy.Symbol('beta')
gamma = sympy.Symbol('gamma')
g0 = sympy.Symbol('g0')
theta = sympy.Symbol('theta')
lamb = sympy.Symbol('lamb')
no_num = sympy.Symbol('no_num')

nis = {}
nis_keys = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            nis[(o,i,j)] = sympy.Symbol('n%d,%d,%d'%(o,i,j))
            nis_keys.append((o,i,j))
miu = {}
for o in origins_input:
    for i in userclass_input[o]:
        miu[(o,i)] = sympy.Symbol('u%d,%d'%(o,i))

# y_observed_dic = {}
# for o in origins_input:
#     y_observed_dic[o] = {}
#     for i in userclass_input[o]:
#         y_observed_dic[o][i] = sympy.Matrix(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)])

y_observed_li = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            y_observed_li.append(train_info['DEPARTURE_RATE_average_%d_%d'%(o,i)][j-1])
y_observed = sympy.Matrix(y_observed_li) 

upper_parameters_value = {}
upper_parameters_value[alpha] = alpha_value_initial
upper_parameters_value[beta] = beta_value_initial
upper_parameters_value[gamma] = gamma_value_initial
upper_parameters_value[g0] = g0_value_initial
upper_parameters_value[theta] = theta_value_initial

In [3]:
flow_nis_aggregate = sympy.zeros(di,len(station_interval))
flow_nis_aggregate_dic = {}
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] = flow_nis_aggregate[j - 1,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(nis[(o,i,j)], (1, origins_interval_num[o])))

for j in train_info['TYPE']:
    for i in station_interval:
        flow_nis_aggregate_dic[(j, i)] = flow_nis_aggregate[(j - 1),(151019789 - i)]  

#车厢内拥挤成本
def invehicle_crowding_cost(origin, train, Tv_dic, n0_dic, flow_dic, g0_para):
    crowding_cost = 0
    for i in range(origin, 151019778, -1):
        crowding_cost = crowding_cost + g0_para * Tv_dic[i] * (flow_dic[(train, i)] + n0_dic[(train, i)] - 256) / 275
    return crowding_cost

In [4]:
#lagrange函数对n的导数
Ln = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            Ln.append((1 / theta) * (sympy.log(nis[(o,i,j)]) + 1) + invehicle_crowding_cost(o, j, Tv, n0, flow_nis_aggregate_dic, g0) + alpha * T[(o,j)] + beta * TE[(o,i,j)] + gamma * TL[(o,i,j)] - miu[(o,i)])            
            
Noi = []
for o in origins_input:
    for i in userclass_input[o]:
        noi = 0
        for j in train_choice_set_input[o][i]:
            noi = noi + nis[(o,i,j)]
        Noi.append(noi) 

#分块矩阵A
A_li = []
for num in range(0,len(Ln)):
    ln = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                ln.append(sympy.diff(Ln[num],nis[(o,i,j)]))
    A_li.append(ln)
    
A = sympy.Matrix(A_li)

#分块矩阵B
B_li = []
for num in range(0,len(Ln)):
    lu = []
    for o in origins_input:
        for i in userclass_input[o]:
            lu.append(sympy.diff(Ln[num],miu[(o,i)]))
    B_li.append(lu)
    
B = sympy.Matrix(B_li)

#分块矩阵C
C_li = []
for num in range(0,len(Noi)):
    nn = []
    for o in origins_input:
        for i in userclass_input[o]:
            for j in train_choice_set_input[o][i]:
                nn.append(sympy.diff(Noi[num],nis[(o,i,j)]))
    C_li.append(nn)  
    
C = sympy.Matrix(C_li)

#分块矩阵D
D_li = []
for num in range(0,len(Noi)):
    nu = []
    for o in origins_input:
        for i in userclass_input[o]:
            nu.append(sympy.diff(Noi[num],miu[(o,i)]))
    D_li.append(nu)        

D = sympy.Matrix(D_li) 

#分块矩阵N
N_li = []
for num in range(0,len(Ln)):
    lp = []
    lp.append(sympy.diff(Ln[num],alpha))
    lp.append(sympy.diff(Ln[num],beta))
    lp.append(sympy.diff(Ln[num],gamma))
    lp.append(sympy.diff(Ln[num],g0))
    lp.append(sympy.diff(Ln[num],theta))
    N_li.append(lp)
    
N = sympy.Matrix(N_li)

In [5]:
#cost_fuction
def train_cost(flow_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para):
    
    train_cost_dic = {}
    for o in origins_input:
        for i in userclass[o]:
            for j in train_choice_set[o][i]:           
                train_cost_dic[(o, i, j)] = invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]

    return train_cost_dic

#logit车次流量加载
def logit(origin, user, train_cost_dic, demand, choice_set, theta_para):
    
    cost_exp_li = []
    for j in train_info['TYPE']:
        if j in choice_set:
            cost_exp_li.append(sympy.exp( - theta_para * train_cost_dic[(origin, user, j)]))
        else:
            cost_exp_li.append(0)
            
    flow = (sympy.Matrix(cost_exp_li) / np.sum(cost_exp_li)) * demand 
    return flow

#多类用户logit加载
def multiorigin_multiclass_logit(userclass, train_cost_dic, demand_dic, train_choice_set, theta_para):
    
    flow_multiorigin_multiclass_aggregate = sympy.zeros(di,len(station_interval))
    flow_multiorigin_multiclass_dic = {}
    flow_multiorigin_multiclass_aggregate_dic = {}
    
    for o in origins_input:
        flow_multiorigin_multiclass_dic[o] = {}
        for i in userclass[o]:
            flow_multiorigin_multiclass_dic[o][i] = logit(o, i, train_cost_dic, demand_dic[o][i], train_choice_set[o][i], theta_para)
            flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] = flow_multiorigin_multiclass_aggregate[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multiorigin_multiclass_dic[o][i], (1, origins_interval_num[o])))
    
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_multiorigin_multiclass_aggregate_dic[(j, i)] = flow_multiorigin_multiclass_aggregate[(j - 1),(151019789 - i)]
    
    return flow_multiorigin_multiclass_aggregate, flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic

#cost_fuction_stochastic
def train_cost_stochastic(flow_multiorigin_multiclass_dic, flow_multiorigin_multiclass_aggregate_dic, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para, theta_para):
    
    train_cost_stochastic_dic = {}
    for o in origins_input:
        train_cost_stochastic_dic[o] = {}
        for i in userclass[o]:
            flow_origin_class_dic = dict(zip(train_info['TYPE'], flow_multiorigin_multiclass_dic[o][i]))
            train_cost_stochastic_dic[o][i] = {}
            for j in train_info['TYPE']:  #保证矩阵运算时宽度一致
                if j in train_choice_set[o][i]:
                    train_cost_stochastic_dic[o][i][j] = (1 / theta_para) * (sympy.log(flow_origin_class_dic[j])+ 1) + invehicle_crowding_cost(o, j, Tv_dic, n0_dic, flow_multiorigin_multiclass_aggregate_dic, g0_para) + beta_para * TE[(o, i, j)] + gamma_para * TL[(o, i, j)] + alpha_para * T[o, j]
                else:
                    train_cost_stochastic_dic[o][i][j] = no_num
                    
    return train_cost_stochastic_dic

#二分法求步长
def binary_search(equation):
    a = 0
    b = 1
    fa = equation.evalf(subs = {lamb:a})
    fb = equation.evalf(subs = {lamb:b})
    while a <= b:
        mid = (a + b) / 2
        fm = equation.evalf(subs = {lamb:mid})
        if abs(fm) < 0.1:
            #print("方程的根为",mid)
            break
        if fa * fm < 0:
            a = a
            b = mid
            fb = equation.evalf(subs = {lamb:b})
            #print("解在[",a,",",b,"]之间")
        elif fb * fm < 0:
            a = mid
            b = b
            fa = equation.evalf(subs = {lamb:a})
            #print("解在[",a,",",b,"]之间")
            
    return mid

In [6]:
def train_flow_assignment(userclass, train_choice_set, multi_demand, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para, theta_para):
    n = 0
    indicator = 1
    flow_dic_initial = {}
    for j in train_info['TYPE']:
        for i in station_interval:
            flow_dic_initial[(j, i)] = 0
    cost_dic = train_cost(flow_dic_initial, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    flow_aggregate_x_new, flow_multi_dic_x_new, flow_aggregate_dic_x_new = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set, theta_para)
    
    while (indicator > 0.00001):
    
        flow_multi_dic_x = flow_multi_dic_x_new
        flow_aggregate_dic_x = flow_aggregate_dic_x_new

        #车次成本更新
        cost_dic = train_cost(flow_aggregate_dic_x, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para)
        #logit流量加载
        flow_aggregate_y, flow_multi_dic_y, flow_aggregate_dic_y = multiorigin_multiclass_logit(userclass, cost_dic, multi_demand, train_choice_set, theta_para)
        
        #辅助流量计算（计算迭代步长）
        flow_aggregate_aux = sympy.zeros(di,len(station_interval))
        flow_multi_dic_aux = {}
        flow_aggregate_dic_aux = {}
        
        for o in origins_input:
            flow_multi_dic_aux[o] = {}
            for i in userclass[o]:
                flow_multi_dic_aux[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb
                flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] = flow_aggregate_aux[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_aux[o][i], (1, origins_interval_num[o])))

        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_aux[(j, i)] = flow_aggregate_aux[(j - 1),(151019789 - i)]
                
        #计算迭代步长
        train_cost_stochastic_aux_dic = train_cost_stochastic(flow_multi_dic_aux, flow_aggregate_dic_aux, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para, theta_para)
        equation_lamb = 0
        for o in origins_input:
            for i in userclass[o]:
                equation_lamb = equation_lamb + ((flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]).T * sympy.Matrix(list(train_cost_stochastic_aux_dic[o][i].values())))[0,0]
        print('start')
        lamb_value_df = binary_search(equation_lamb)
        print(lamb_value_df)
    
        #确定新的迭代起点
        flow_multi_dic_x_new = {}
        flow_aggregate_dic_x_new = {}
        flow_aggregate_x_new = sympy.zeros(di,len(station_interval))
        for o in origins_input:
            flow_multi_dic_x_new[o] = {}
            for i in userclass[o]:
                flow_multi_dic_x_new[o][i] = flow_multi_dic_x[o][i] + (flow_multi_dic_y[o][i] - flow_multi_dic_x[o][i]) * lamb_value_df
                flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] = flow_aggregate_x_new[:,(11 - origins_interval_num[o]):11] + sympy.Matrix(np.tile(flow_multi_dic_x_new[o][i], (1, origins_interval_num[o])))
    
        for j in train_info['TYPE']:
            for i in station_interval:
                flow_aggregate_dic_x_new[(j, i)] = flow_aggregate_x_new[(j - 1),(151019789 - i)]
        
        #收敛性检验
        numerator = 0
        denominator = 0
        for o in origins_input:
            for i in userclass[o]:
                numerator = numerator + ((flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]).T * (flow_multi_dic_x_new[o][i] - flow_multi_dic_x[o][i]))[0,0]
                denominator = denominator + (sympy.ones(1,di) * flow_multi_dic_x[o][i])[0,0]
        
        indicator = (numerator ** 0.5) / denominator
        n = n + 1
        print(indicator)
        print(n)
        
    cost_dic_x_new = train_cost(flow_aggregate_dic_x_new, userclass, train_choice_set, Tv_dic, TE_dic, TL_dic, n0_dic, alpha_para, beta_para, gamma_para, g0_para) 
    
    return flow_multi_dic_x_new, flow_aggregate_x_new, flow_aggregate_dic_x_new, cost_dic_x_new

In [7]:
flow_multi_dic, flow_aggregate, flow_aggregate_dic, cost_dic = train_flow_assignment(userclass_input, train_choice_set_input, demand_input, Tv, TE, TL, n0, upper_parameters_value[alpha], upper_parameters_value[beta], upper_parameters_value[gamma], upper_parameters_value[g0], upper_parameters_value[theta])

# upper_obj = 0
# for o in origins_input:
#     for i in userclass_input[o]:
#         upper_obj = upper_obj + ((y_observed_dic[o][i] - flow_multi_dic[o][i]).T * (y_observed_dic[o][i] - flow_multi_dic[o][i]))[0,0]

# upper_obj_record.append(upper_obj)    

nis_value_dic = {}
nis_value_li = []
for o in origins_input:
    for i in userclass_input[o]:
        for j in train_choice_set_input[o][i]:
            nis_value_dic[nis[(o,i,j)]]= flow_multi_dic[o][i][j - 1,0]
            nis_value_li.append(flow_multi_dic[o][i][j - 1,0])         

nis_value_matrix = sympy.Matrix(nis_value_li)

#上层目标值记录
upper_obj_record.append(sympy.simplify(((y_observed - nis_value_matrix).T * (y_observed - nis_value_matrix))[0,0]))
print(f'上层目标值均衡:{upper_obj_record[-1]}')
                          
#矩阵运算所有参数取值
matrix_parameters_value_all = {}
matrix_parameters_value_all.update(nis_value_dic)
matrix_parameters_value_all.update(upper_parameters_value)

#矩阵计算
A_value = np.matrix(A.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
B_value = np.matrix(B.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
C_value = np.matrix(C.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
D_value = np.matrix(D.evalf(subs = matrix_parameters_value_all)).astype(np.float64)
N_value = np.matrix(N.evalf(subs = matrix_parameters_value_all)).astype(np.float64)

#梯度计算
I11_value = A_value.I + A_value.I * B_value * (D_value - C_value * A_value.I * B_value).I * C_value * A_value.I
Grad = - I11_value * N_value

start
0.96875
0.000226995519724849
1
start
0.5
2.63555204293814e-6
2
上层目标值均衡:0.00184451709208375


In [11]:
Grad[:,1]

matrix([[-18.35585653],
        [-22.9504255 ],
        [-34.29817341],
        [-33.15088537],
        [-44.90435678],
        [-44.03108617],
        [-51.31407803],
        [-23.39074122],
        [ 15.50155919],
        [ 72.57729201],
        [ 53.9702672 ],
        [ 41.45824205],
        [ 33.20005395],
        [ 19.76349461],
        [ 12.13114753],
        [  8.83661025],
        [  6.94582126],
        [  4.5812038 ],
        [  3.42991115],
        [-19.48027495],
        [-20.23111616],
        [-24.08044236],
        [-22.8643121 ],
        [-23.02756657],
        [-28.62080171],
        [-27.44609138],
        [-20.65737146],
        [-10.14934528],
        [  7.77342644],
        [ 59.4427602 ],
        [ 37.08977595],
        [ 35.81331443],
        [ 17.06382032],
        [ 12.27749166],
        [  9.41719152],
        [  5.17732923],
        [  3.97680026],
        [  3.18374209],
        [  1.89511235],
        [  1.1640462 ],
        [  0.84752019],
        [  0.666

In [12]:
# 反应函数
s = 0.01
y = nis_value_matrix + Grad * sympy.Matrix([0, s, 0, 0, 0])

In [13]:
y

Matrix([
[ 0.694402503821443],
[  1.09535531774341],
[  1.98155173741088],
[  2.20767393879974],
[  4.32973176423231],
[  6.05814611719634],
[   8.9712415024628],
[  13.7292123618727],
[   21.152711781201],
[  29.1453423048617],
[  18.3521024661573],
[   14.091744669783],
[  11.2646505120578],
[  6.70297323257854],
[  4.13205796932526],
[   3.0010564854264],
[  2.36365182379282],
[  1.55959105312432],
[  1.16680245815236],
[ 0.822542181187861],
[ 0.955288089122146],
[  1.28850179216427],
[  1.30454201496278],
[  1.80770128962774],
[  3.09629957324722],
[  3.56344702772303],
[  5.58490208412591],
[     10.0599503029],
[  11.1774966967678],
[  17.1519838183026],
[  10.6392060503679],
[  10.2761971626277],
[  4.89564579833717],
[  3.53562840050033],
[    2.736470841933],
[  1.49554273220862],
[  1.14835919422095],
[ 0.917974140455678],
[ 0.546235908717179],
[ 0.336728142910109],
[  0.24456082481641],
[  0.19261780420322],
[ 0.127093561281068],
[0.0950845672892465],
[ 0.163338114489097],
[

In [14]:
nis_value_matrix

Matrix([
[ 0.877961069095739],
[  1.32485957277468],
[  2.32453347146379],
[  2.53918279249792],
[  4.77877533204988],
[   6.4984569788989],
[  9.48438228277861],
[  13.9631197740822],
[  20.9976961892699],
[  28.4195693847701],
[  17.8123997941387],
[  13.6771622492734],
[  10.9326499725126],
[  6.50533828652214],
[  4.01074649402118],
[  2.91269038290286],
[  2.29419361115674],
[  1.51377901512775],
[  1.13250334666298],
[  1.01734493070069],
[  1.15759925070364],
[  1.52930621572614],
[  1.53318513598394],
[  2.03797695529434],
[  3.38250759034513],
[  3.83790794156338],
[  5.79147579869966],
[  10.1614437556794],
[  11.0997624323431],
[   16.557556216347],
[  10.2683082908517],
[  9.91806401832132],
[   4.7250075951523],
[   3.4128534839047],
[  2.64229892674766],
[  1.44376943987473],
[  1.10859119157843],
[ 0.886136719587366],
[ 0.527284785220811],
[ 0.325087680869179],
[ 0.236085622933268],
[ 0.185953923569765],
[ 0.122698039294274],
[0.0917940587080483],
[ 0.221541540102283],
[

In [15]:
flow_multi_dic, flow_aggregate, flow_aggregate_dic, cost_dic = train_flow_assignment(userclass_input, train_choice_set_input, demand_input, Tv, TE, TL, n0, upper_parameters_value[alpha], upper_parameters_value[beta] + 0.01, upper_parameters_value[gamma], upper_parameters_value[g0], upper_parameters_value[theta])

start
0.96875
0.000230129924894564
1
start
0.5
2.63264155642730e-6
2


In [16]:
flow_multi_dic

{151019789: {0: Matrix([
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [0.711504922639699],
  [  1.1128658385457],
  [ 2.00338988443033],
  [ 2.22589252462223],
  [ 4.34530087917666],
  [ 6.06587915252462],
  [ 8.97476104210984],
  [ 13.7158093223744],
  [ 21.1297187931994],
  [ 29.1222184732698],
  [ 18.3397711617279],
  [ 14.0822907405552],
  [  11.257099253216],
  [ 6.69850391770133],
  [ 4.12929191230704],
  [ 2.99905505153388],
  [  2.3620746013791],
  [ 1.55854844349745],
  [ 1.16602408518943],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [                0],
  [      

In [28]:
invehicle_crowding_cost(151019789, 1, Tv, n0, flow_nis_aggregate_dic, g0)

190*g0*Max(0, n151019784,4,1/275 - 0.649939393927273) + 180*g0*Max(0, n151019784,4,1/275 - 0.437818181818182) + 180*g0*Max(0, n151019784,4,1/275 + 0.145939393818182) + 190*g0*Max(0, n151019784,4,1/275 + 0.903757575636364) + 190*g0*Max(0, n151019784,4,1/275 + 1.45042424254545) + 140*g0*Max(0, n151019784,4,1/275 + 1.51854545454545)

In [37]:
(n0[(24,151019779)] - 256)/275

4.248969698181818

In [23]:
train_choice_set_input

{151019789: {0: [11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29],
  1: [5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29],
  2: [19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44],
  3: [16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39]},
 151019787: {0: [14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32],
  1: [11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29],
  2: [21,
   22,
   23,
   24,
   

In [9]:
sympy.Max(beta**3, 0)

Max(0, beta**2)

In [18]:
(sympy.diff(sympy.Max(beta**3, 0), beta)).evalf(subs = {beta:2})

12.0000000000000

In [11]:
(sympy.diff(beta**2, beta)).evalf(subs = {beta:2})

4.00000000000000

In [23]:
sympy.log(0.00000000000000000000000000000000000000000000000000000000000000000000000000000001)

-184.206807439524

In [24]:
np.log(0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000001)

-200.32490309048197